# Single Source Shortest Path (SSSP) - Santa Clara

In this notebook, we will use cuGraph to compute the shortest path from a starting vertex to everyother vertex in our training dataset.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 05/01/2019
* Modified by John Murray
* Last Edit: 05/29/2019

RAPIDS Versions: 0.7.0 - Docker Container    

Test Hardware

* GV100 32G, CUDA 10.1, Ubuntu 18.04



## Introduction

Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices. 

To compute SSSP for a graph in cuGraph we use:
**cugraph.sssp(G, source)**

Input
* __G__: cugraph.Graph object
* __source__: int, Index of the source vertex

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['distance']: The computed distance from the source vertex to this vertex


## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    
    

### Test Data
We will be using the US Census Bureau, Department of Commerce Tiger Line Roads dataset, for Santa Clara County, converted into a graph.  
A sample is shown below:

![Road_Graph](./img/santa_clara_road_graph.png)

This is a weighted graph, using the edge distance in yards.
The raw data source does not contain nodes. These have been derived through Spatial Inference using Fusion Data Science Spatia software.  
Because the raw data does not contain information about one way roads and restricted access intersections, assumptions have been made and therefore the data should not be used for navigation purposes.
__Note__: The derived dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  
Base image: USGS Imagery Topo Base Map: Map services and data available from U.S. Geological Survey, National Geospatial Program  
Licensed under [U.S. Government Work License](https://www.usa.gov/government-works)

### Read the source data and adjust the vertex IDs

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [2]:
# Test file  - Read the Santa Clara node table as a graph. (Store dataset in ./data folder)
datafile='./data/santa_clara_road_graph_20190526.csv'

In [3]:
# Read the data file with lenght of edge in metres as weight
cols = ["src", "dst","length"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("length", "float32"),
        ])

gdf = cudf.read_csv(datafile, names=cols, dtype=list(dtypes.values()) ,skiprows=1)

In [4]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [5]:
# Display the results
print(gdf)

      src    dst     length  src_0  dst_0
0    568    567  709.93207    567    566
1    567    568  709.93207    566    567
2  39732  65418    33.2337  39731  65417
3  65418  39732    33.2337  65417  39731
4  64991  64992  138.18001  64990  64991
5  64992  64991  138.18001  64991  64990
6  69015  69016    371.529  69014  69015
7  69016  69015    371.529  69015  69014
8  61591  61592  600.34796  61590  61591
9  61592  61591  600.34796  61591  61590
[188248 more rows]


### Create the Road Graph and call SSSP

In [6]:
%%time
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"],gdf['length'])

CPU times: user 170 µs, sys: 156 µs, total: 326 µs
Wall time: 333 µs


In [7]:
%%time
# Call cugraph.sssp to get the distances from vertex 51096 (NVIDIA Endeavor HQ, Santa Clara):
df = cugraph.sssp(G,51096)

CPU times: user 539 ms, sys: 124 ms, total: 663 ms
Wall time: 662 ms


In [8]:
print(df)

    vertex       distance
0       0       73044.68
1       1      74360.516
2       2      74383.234
3       3  3.4028235e+38
4       4        76425.3
5       5      77057.836
6       6  3.4028235e+38
7       7  3.4028235e+38
8       8  3.4028235e+38
9       9  3.4028235e+38
[74354 more rows]


In [9]:
# Exclude non-connected nodes set to fp32 max
print(df.query("distance<3.4028234e+38"))

    vertex   distance
0       0   73044.68
1       1  74360.516
2       2  74383.234
4       4    76425.3
5       5  77057.836
17      17   70101.27
18      18   72735.48
19      19    71538.6
20      20   75988.56
22      22   72687.13
[73800 more rows]


### Calculate nearest and farthest vertices

In [10]:
# Get 10 nearest vertices, excluding non-connected
print(df.query("distance<3.4028234e+38").nsmallest(10,['distance']))

    vertex   distance
51096   51096        0.0
51095   51095    14.5925
50996   50996     37.778
51111   51111  140.13301
8869    8869  155.71552
65316   65316    162.967
51098   51098  166.22672
8868    8868   177.2935
51097   51097  182.15302
57681   57681   189.4866


In [11]:
# Get 10 farthest vertices, excluding non-connected
print(df.query("distance<3.4028234e+38").nlargest(10,['distance']))

    vertex    distance
289     289  104718.984
295     295   103530.08
544     544   102600.29
545     545   102391.88
72286   72286   101753.05
474     474   100851.62
72288   72288   100300.77
546     546    100205.1
543     543    99003.07
542     542     97302.0


In [12]:
# Need to shift the vertex IDs back to start with one rather than zero (next version of cuGraph will fix this issue)
df["vertex"] = df["vertex"] + 1

In [13]:
# Output vertex distance vector to csv for display in GIS software
df.query("distance<3.4028234e+38").to_pandas().to_csv("./data/santa_clara_node_distances.csv", index=False, header=True, sep=',')

___
Copyright (c) 2019, NVIDIA CORPORATION.

Adapted by John Murray

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___